In [1]:
#libraries needed
import pandas as pd
import snscrape.modules.twitter as sntwitter
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
# nltk.download('stopwords') #run once and comment it out to avoid it downloading multiple times
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

import string
import re
import textblob
from textblob import TextBlob

from wordcloud import WordCloud, STOPWORDS
from emot.emo_unicode import UNICODE_EMOJI


porter = PorterStemmer()

lemmatizer = WordNetLemmatizer() 

from wordcloud import ImageColorGenerator
from PIL import Image

import warnings


In [4]:
df = pd.read_csv('dataset_twitter.csv',encoding='ISO-8859-1')
df.head()

,"ï»¿""conversation_id""",created_at,favorite_count,full_text,hashtags/0,hashtags/1,hashtags/2,hashtags/3,hashtags/4,hashtags/5,...,user_mentions/14/screen_name,user_mentions/15/id_str,user_mentions/15/name,user_mentions/15/profile,user_mentions/15/screen_name,user_mentions/16/id_str,user_mentions/16/name,user_mentions/16/profile,user_mentions/16/screen_name,view_count
0,1621155929533337609,2023-02-02T14:37:47.000Z,6,Calling all accountants and bookkeepers lookin...,virtualfinance,Accounting,bookkeeping,ebook,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,116129.0
1,1656947523213303810,2023-05-12T09:00:48.000Z,3,Our wonderful partners @LinfordGrey showing ho...,payroll,payments,accounting,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122.0
2,1654499440172883969,2023-05-05T14:53:00.000Z,0,RT @wolterskluweruk: We have a brilliant panel...,inclusion,equity,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,206.0
3,1651865485732978689,2023-04-28T08:26:36.000Z,0,RT @LinfordGrey: We're a digital-first practic...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,886.0
4,1651154177748262913,2023-04-26T09:20:07.000Z,4,We're exhibiting at @Accountex London 2023! â­...,AccountexLondon,accounting,finance,payments,bookkeeping,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,161.0


**Data** **Wrangling**

**Data PreProcessing**

In [5]:
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')
eng_stop_words = list(stopwords.words('english'))


[nltk_data] Downloading package stopwords to /Users/Amara/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Amara/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Amara/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
emoji = list(UNICODE_EMOJI.keys())


In [7]:
# function for preprocessing tweet in preparation for sentiment analysis
def ProcessedTweets(text):
    #changing tweet text to small letters
    text = text.lower()
    # Removing @ and links 
    text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", text).split())
    # removing repeating characters
    text = re.sub(r'\@\w+|\#\w+|\d+', '', text)
    # removing punctuation and numbers
    punct = str.maketrans('', '', string.punctuation+string.digits)
    text = text.translate(punct)
    # tokenizing words and removing stop words from the tweet text
    tokens = word_tokenize(text)  
    filtered_words = [w for w in tokens if w not in eng_stop_words]
    filtered_words = [w for w in filtered_words if w not in emoji]
    # lemmetizing words
    lemmatizer = WordNetLemmatizer() 
    lemma_words = [lemmatizer.lemmatize(w) for w in filtered_words]
    text = " ".join(lemma_words)
    return text

In [8]:
# Generate a new column called 'Processed Tweets' by applying preprocessed tweets function to the 'Tweet' column.
#df['Processed_Tweets'] = df['tweet'].apply(ProcessedTweets)
df['Processed_Tweets'] = df['full_text'].apply(ProcessedTweets)



In [9]:
df.head(5)

,"ï»¿""conversation_id""",created_at,favorite_count,full_text,hashtags/0,hashtags/1,hashtags/2,hashtags/3,hashtags/4,hashtags/5,...,user_mentions/15/id_str,user_mentions/15/name,user_mentions/15/profile,user_mentions/15/screen_name,user_mentions/16/id_str,user_mentions/16/name,user_mentions/16/profile,user_mentions/16/screen_name,view_count,Processed_Tweets
0,1621155929533337609,2023-02-02T14:37:47.000Z,6,Calling all accountants and bookkeepers lookin...,virtualfinance,Accounting,bookkeeping,ebook,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,116129.0,calling accountant bookkeeper looking delight ...
1,1656947523213303810,2023-05-12T09:00:48.000Z,3,Our wonderful partners @LinfordGrey showing ho...,payroll,payments,accounting,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122.0,wonderful partner showing easy pay employee ð...
2,1654499440172883969,2023-05-05T14:53:00.000Z,0,RT @wolterskluweruk: We have a brilliant panel...,inclusion,equity,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,206.0,rt brilliant panel covering next week elise sa...
3,1651865485732978689,2023-04-28T08:26:36.000Z,0,RT @LinfordGrey: We're a digital-first practic...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,886.0,rt digitalfirst practice look like like weve m...
4,1651154177748262913,2023-04-26T09:20:07.000Z,4,We're exhibiting at @Accountex London 2023! â­...,AccountexLondon,accounting,finance,payments,bookkeeping,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,161.0,exhibiting london â­ï¸ book free ticket toda...


**Sentiment Analysis**

In [10]:
# Function for polarity score
def polarity(tweet):
    return TextBlob(tweet).sentiment.polarity

def subjectivity(tweet):
    return TextBlob(tweet).sentiment.subjectivity

# Function to get sentiment type
#setting the conditions
def sentimenttextblob(polarity):
    if polarity < 0:
        return "Negative"
    elif polarity == 0:
        return "Neutral"
    else:
        return "Positive" 

In [11]:
# using the functions to get the polarity and sentiment
df['Polarity'] = df['Processed_Tweets'].apply(polarity)
df['Subjectivity'] = df['Processed_Tweets'].apply(subjectivity)

df['Sentiment'] = df['Polarity'].apply(sentimenttextblob)

sent = df['Sentiment'].value_counts()
sent

Positive    117
Neutral      92
Negative     13
Name: Sentiment, dtype: int64

In [12]:
df.head(2)

,"ï»¿""conversation_id""",created_at,favorite_count,full_text,hashtags/0,hashtags/1,hashtags/2,hashtags/3,hashtags/4,hashtags/5,...,user_mentions/15/screen_name,user_mentions/16/id_str,user_mentions/16/name,user_mentions/16/profile,user_mentions/16/screen_name,view_count,Processed_Tweets,Polarity,Subjectivity,Sentiment
0,1621155929533337609,2023-02-02T14:37:47.000Z,6,Calling all accountants and bookkeepers lookin...,virtualfinance,Accounting,bookkeeping,ebook,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,116129.0,calling accountant bookkeeper looking delight ...,0.300000,0.800000,Positive
1,1656947523213303810,2023-05-12T09:00:48.000Z,3,Our wonderful partners @LinfordGrey showing ho...,payroll,payments,accounting,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,122.0,wonderful partner showing easy pay employee ð...,0.716667,0.916667,Positive


In [20]:
X = df['Processed_Tweets'].values
y = df['Sentiment'].values